## read me

This file can process both resting state data and event related potentials. For the second, it's important that the data has event markers created with the script in the bitbrain importing section. 

Set variable **erp** to true or false to determine if you want to process event related potentials (True) or resting state data (false)

In [54]:
%load_ext autoreload
%autoreload 2
%matplotlib qt

import numpy as np
import csv
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob
from tqdm import tqdm 
from atpbar import atpbar
from datetime import datetime
import mne
import autoreject
from mne.preprocessing import ICA, corrmap, create_ecg_epochs, create_eog_epochs

from pyprep.find_noisy_channels import NoisyChannels
from mne_icalabel import label_components


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
def raw_to_events(raw):
    event_id = {'-1':-1, '1':1, '-2':2, '2':2, '-3':-3, '3':3}
    events = mne.events_from_annotations(raw, event_id=event_id)[0]
    if len(events)==0:
        events = np.zeros((1,3))
        events = events.astype(int)
    return events 

In [ ]:
for erp in [False]:
    if erp == True: 
        paths = glob.glob("../../data/mne_raw_events/*")
        paths.sort()
        keywords = ["_mmn-raw_phot-events.fif"]
        paths = [path for path in paths if any(keyword in path for keyword in keywords)]
    else:
        paths = glob.glob("../../data/mne_raw/*")
        paths.sort()
        keywords = ["mmn-raw", "ltp-raw", "nimitta-raw", 'rmmn-raw', 'tetanus-raw']
        paths = [path for path in paths if not any(keyword in path for keyword in keywords)]
        paths = paths[111:]
    
    
    df = pd.DataFrame(columns=[
        'path_name',
        'bad_ch', 'bad_ch_count', 'ica_removed', 
        'ica_removed_count', 'epochs_original', 
        'epochs_remaining', 'epochs_rejected',
        'epochs_rejected_percentage'
    ])
    
    try:
        df = pd.read_csv('../../data/AR_ICA_cleaning_results.csv')
    except: 
        pass
    
    
    for path in paths:
        
        raw = mne.io.read_raw_fif(path, preload=True)
        montage = mne.channels.make_standard_montage('standard_1020')
        raw.set_montage(montage)
        
        filt_raw = raw.copy()
        filt_raw.filter(0.3, 45)
        filt_raw.notch_filter(freqs=[60,76, 120])
        filt_raw.notch_filter(freqs=84, notch_widths=1, phase='zero')
    
        
        nd = NoisyChannels(filt_raw, random_state=1337)
        #nd.find_bad_by_correlation()
        nd.find_bad_by_deviation()
        nd.find_bad_by_ransac()
        #bads = nd.bad_by_correlation+nd.bad_by_deviation+nd.bad_by_ransac
        bads = nd.bad_by_deviation+nd.bad_by_ransac
        filt_raw.info['bads'] = bads
    
        if erp == True:
            events = raw_to_events(raw)    
            filt_epochs = mne.Epochs(filt_raw, events, tmin=-0.5, tmax=1.5, baseline=(-0.1, 0), preload=True)
            
        else:
            filt_epochs = mne.make_fixed_length_epochs(filt_raw, duration=3, overlap=0, preload=True)

        if len (filt_epochs)<10: 
            print (f'!!!!!!! too few epochs: {path}')
            continue
            
        
        filt_epochs_orig = filt_epochs.copy()
        
        ar = autoreject.AutoReject(n_interpolate=[1, 2, 3, 4], random_state=11, n_jobs=1, verbose=True)
        ar.fit(filt_epochs)
        epochs_ar, reject_log = ar.transform(filt_epochs, return_log=True)
        
        #filt_epochs[reject_log.bad_epochs].plot(scalings=dict(eeg=100e-6))
        #reject_log.plot('horizontal')
        
        ica = ICA(
            n_components=15,
            max_iter="auto",
            method="infomax",
            random_state=97,
            fit_params=dict(extended=True),
        )
        
        try:
            ica.fit(filt_epochs[~reject_log.bad_epochs])
        except:
            continue
        ic_labels = label_components(filt_epochs, ica, method="iclabel")
        labels = ic_labels["labels"]
        
        exclude_idx = [
            idx for idx, label in enumerate(labels) if label not in ["brain", "other"]
        ]   
            
        ica.apply(filt_epochs, exclude=exclude_idx)
        
        
        #ica.plot_sources(filt_epochs)
        #ica.plot_components()
        #ica.plot_properties(raw, picks=[0, 12], verbose=False)
        #filt_epochs_orig.plot()
        #filt_epochs.plot()
        
        ar = autoreject.AutoReject(n_interpolate=[1, 2, 3, 4], random_state=11, n_jobs=1, verbose=True)
        ar.fit(filt_epochs)
        epochs_ar_2, reject_log_2 = ar.transform(filt_epochs, return_log=True)
        
        #filt_epochs[reject_log.bad_epochs].plot(scalings=dict(eeg=100e-6))
        #reject_log2.plot('horizontal')
    
        print ("##############")
        print (path)
        
        print (f"bad channels: {bads}")
        for i in ic_labels["labels"]:
            print (i)
        print(f"Excluding these ICA components: {exclude_idx}")
        print(f"remaining epochs: {len(epochs_ar_2)}")
        
        plotting = False
        if plotting == True:
            ica.plot_components()
    
            reject_log.plot('horizontal')
            reject_log_2.plot('horizontal')
    
            filt_epochs_orig.plot(title="original")
            filt_epochs.plot(title="filt_epochs")
            epochs_ar_2.plot(title="epochs_ar_2")
    
            #epochs_ar_2.drop_log
    
        #SAVING
        basename = os.path.basename(path)
        folder = os.path.dirname(path)
        output_fname = f"{folder}_epochs_cleaned/{basename}"
        epochs_ar_2.save(output_fname, overwrite=True)
        
    
        # save df
       
        bad_ch = str(bads)
        bad_ch_count = len(bads)
        ica_removed = str(exclude_idx)
        ica_removed_count = len(exclude_idx)
        epochs_original = len(filt_epochs_orig)
        epochs_remaining = len(epochs_ar_2)
        epochs_rejected = len(filt_epochs_orig) - len(epochs_ar_2)
        epochs_rejected_percentage = np.round((1-(epochs_remaining/epochs_original)),2)
        
        # Create a new DataFrame for this iteration
        new_row = pd.DataFrame({
            'path_name': [os.path.basename(path)],
            'bad_ch': [bad_ch], 'bad_ch_count': [bad_ch_count], 'ica_removed': [ica_removed],
            'ica_removed_count': [ica_removed_count], 'epochs_original': [epochs_original],
            'epochs_remaining': [epochs_remaining], 'epochs_rejected': [epochs_rejected], 
            'epochs_rejected_percentage': [epochs_rejected_percentage]
        })
        
        df = pd.concat([df, new_row], ignore_index=True)
    
        df.to_csv('../../data/AR_ICA_cleaning_results.csv', index=False)
    


Opening raw data file ../../data/mne_raw/sub7-day1-mindfulness-raw.fif...
    Range : 256 ... 390671 =      1.000 ...  1526.059 secs
Ready.
Reading 0 ... 390415  =      0.000 ...  1525.059 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.3 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 2817 samples (11.004 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming wind

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 83 - 85 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 83.00
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 82.75 Hz)
- Upper passband edge: 85.00 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 85.25 Hz)
- Filter length: 1691 samples (6.605 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


...

  0%|          |  : 0/303 [00:00<?,       ?it/s]


RANSAC done!
Not setting metadata
505 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 505 events and 768 original time points ...
0 bad epochs dropped
Running autoreject on ch_type=eeg


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/autoreject/utils.py:73: UserWarning: 1 channels are marked as bad. These will be ignored. If you want them to be considered by autoreject please remove them from epochs.info["bads"].
  warnings.warn(


  0%|          | Creating augmented epochs : 0/31 [00:00<?,       ?it/s]

  0%|          | Computing thresholds ... : 0/31 [00:00<?,       ?it/s]